**Created on 04/01/2023**

**Authors: Thomas ROSTAINGT**

**License: this code is released under the CeCILL 2.1 license. See https://www.cecill.info/licenses/Licence_CeCILL_V2.1-en.txt**

In [1]:
# libraries pour créer des exemples
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

## Importation des données

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/M2 SID/PIP2023/Dataset_complete.csv')

In [ ]:
df.head()

,initiator,target,protocol,nlos_indicator,t1,t2,t3,t4,skew,tof,...,course_dist,temperature_initiator,temperature_target,timestamp,seqnum,rssi_request,rssi_ack,rssi_data,rssi_mean,register_dump_CIR
0,183,100,TWR,2.783982,946218857614,946490292658,946594234510,946322802711,-2.368804,1622,...,0.0,30.98,30.220001,1654929794228,4,-80.136574,-80.077130,-79.836296,-80.016667,"{""ACC_MEM"": ""AAT/9gACAAr/+wABAAL/8wAS//AACgAGA..."
1,183,100,TWR,2.920485,1009474709646,1009746235572,1009850439822,1009578917162,-1.045965,1633,...,0.0,30.98,29.840002,1654929795219,10,-80.267769,-79.988684,-79.835720,-80.030724,"{""ACC_MEM"": ""//gAEwAC//X/9gAXABEACwASABQAFgAG/..."
2,183,100,TWR,2.536090,1072923068046,1073194681670,1073299179662,1073027569304,-0.799856,1633,...,0.0,30.98,29.840002,1654929796210,16,-79.964310,-79.850138,-79.693502,-79.835983,"{""ACC_MEM"": ""//QABQAJ//X/9gAUABMADv/3//r//gAK/..."
3,183,100,TWR,2.857669,36731534990,37003232943,37107270798,36835576116,-0.984438,1635,...,0.0,30.98,29.840002,1654929797201,22,-80.040833,-79.818313,-79.849525,-79.902890,"{""ACC_MEM"": ""ACb//AAa//4ADQAJAAYALf/+ABQAEgAOA..."
4,183,100,TWR,2.263103,99923195534,100194974620,100298828942,100027053147,-0.769092,1645,...,0.0,30.98,29.840002,1654929798190,28,-79.999908,-80.102552,-80.022316,-80.041592,"{""ACC_MEM"": ""//b/+P/6//3/8QAI//H//P/1AAkAAv/3A..."


In [ ]:
df.columns

Index(['initiator', 'target', 'protocol', 'nlos_indicator', 't1', 't2', 't3',
       't4', 'skew', 'tof', 'tof_skew', 'range', 'range_skew', 'ranging_unit',
       'ranging_error', 'loc_initiator_x', 'loc_initiator_y',
       'loc_initiator_z', 'loc_target_x', 'loc_target_y', 'loc_target_z',
       'distance', 'course_dist', 'temperature_initiator',
       'temperature_target', 'timestamp', 'seqnum', 'rssi_request', 'rssi_ack',
       'rssi_data', 'rssi_mean', 'register_dump_CIR'],
      dtype='object')

In [ ]:
len(df)

3946

In [4]:
capteurs = np.unique(df['target'])
for capt in capteurs:
  print('nombre de signaux reçus par le capeur',capt,':',len(df[df.target==capt]['seqnum']))
# le nombre de signaux reçus par les capteurs n'est pas le même
# certains capteurs n'ont pas reçu le signal émis par le client 

nombre de signaux reçus par le capeur 100 : 643
nombre de signaux reçus par le capeur 101 : 706
nombre de signaux reçus par le capeur 150 : 477
nombre de signaux reçus par le capeur 171 : 704
nombre de signaux reçus par le capeur 180 : 708
nombre de signaux reçus par le capeur 184 : 708


### Calcul de position 

In [ ]:
df.head()

,initiator,target,protocol,nlos_indicator,t1,t2,t3,t4,skew,tof,...,course_dist,temperature_initiator,temperature_target,timestamp,seqnum,rssi_request,rssi_ack,rssi_data,rssi_mean,register_dump_CIR
0,183,100,TWR,2.783982,946218857614,946490292658,946594234510,946322802711,-2.368804,1622,...,0.0,30.98,30.220001,1654929794228,4,-80.136574,-80.077130,-79.836296,-80.016667,"{""ACC_MEM"": ""AAT/9gACAAr/+wABAAL/8wAS//AACgAGA..."
1,183,100,TWR,2.920485,1009474709646,1009746235572,1009850439822,1009578917162,-1.045965,1633,...,0.0,30.98,29.840002,1654929795219,10,-80.267769,-79.988684,-79.835720,-80.030724,"{""ACC_MEM"": ""//gAEwAC//X/9gAXABEACwASABQAFgAG/..."
2,183,100,TWR,2.536090,1072923068046,1073194681670,1073299179662,1073027569304,-0.799856,1633,...,0.0,30.98,29.840002,1654929796210,16,-79.964310,-79.850138,-79.693502,-79.835983,"{""ACC_MEM"": ""//QABQAJ//X/9gAUABMADv/3//r//gAK/..."
3,183,100,TWR,2.857669,36731534990,37003232943,37107270798,36835576116,-0.984438,1635,...,0.0,30.98,29.840002,1654929797201,22,-80.040833,-79.818313,-79.849525,-79.902890,"{""ACC_MEM"": ""ACb//AAa//4ADQAJAAYALf/+ABQAEgAOA..."
4,183,100,TWR,2.263103,99923195534,100194974620,100298828942,100027053147,-0.769092,1645,...,0.0,30.98,29.840002,1654929798190,28,-79.999908,-80.102552,-80.022316,-80.041592,"{""ACC_MEM"": ""//b/+P/6//3/8QAI//H//P/1AAkAAv/3A..."


### Regroupement des signaux 

In [5]:
#df['seqnum'].iloc[i:i+6]
i=1
df[df.seqnum<i+5]['seqnum'].iloc[i:]

643     5
2530    2
3238    1
Name: seqnum, dtype: int64

In [ ]:
v = 'signal'+str(2)

'signal2'

Méthode analytique de calcul

In [ ]:
df.keys()

Index(['initiator', 'target', 'protocol', 'nlos_indicator', 't1', 't2', 't3',
       't4', 'skew', 'tof', 'tof_skew', 'range', 'range_skew', 'ranging_unit',
       'ranging_error', 'loc_initiator_x', 'loc_initiator_y',
       'loc_initiator_z', 'loc_target_x', 'loc_target_y', 'loc_target_z',
       'distance', 'course_dist', 'temperature_initiator',
       'temperature_target', 'timestamp', 'seqnum', 'rssi_request', 'rssi_ack',
       'rssi_data', 'rssi_mean', 'register_dump_CIR'],
      dtype='object')

In [3]:
def trackObject(x1,y1,r1,x2,y2,r2,x3,y3,r3):
  A = 2*x2 - 2*x1
  B = 2*y2 - 2*y1
  C = r1**2 - r2**2 - x1**2 + x2**2 - y1**2 + y2**2
  D = 2*x3 - 2*x2
  E = 2*y3 - 2*y2
  F = r2**2 - r3**2 - x2**2 + x3**2 - y2**2 + y3**2

  x = (C*E - F*B) / (E*A - B*D)
  y = (C*D - A*F) / (B*D - A*E)
  return x,y

In [7]:
import random

df2 = pd.DataFrame(columns=['seqnum1', 'x1', 'y1', 'r1', 'seqnum2', 'x2', 'y2', 'r2', 'seqnum3', 'x3', 'y3', 'r3', 'xTrue', 'yTrue', 'xCalc', 'yCalc'])

for i in range(len(df)):
    
    if (i%6 == 0):
        
        option1, option2, option3 = random.sample(range(i+1, i+7), 3)
        # vérifier que tous les seqnum sont présents dans le dataframe
        
        OK = False
        while OK == False:
            try : 
                r1 = float(df[df.seqnum == option1].distance)
                r2 = float(df[df.seqnum == option2].distance)
                r3 = float(df[df.seqnum == option3].distance)
                OK = True
            except:
                option1, option2, option3 = random.sample(range(i+1, i+7), 3)

        x1 = float(df[df.seqnum == option1].loc_target_x)
        y1 = float(df[df.seqnum == option1].loc_target_y)  
        x2 = float(df[df.seqnum == option2].loc_target_x)
        y2 = float(df[df.seqnum == option2].loc_target_y)  
        x3 = float(df[df.seqnum == option3].loc_target_x)
        y3 = float(df[df.seqnum == option3].loc_target_y) 
        xTrue = float(df[df.seqnum == option1].loc_initiator_x)
        yTrue = float(df[df.seqnum == option1].loc_initiator_y)
        
        xCalc, yCalc = trackObject(x1,y1,r1,x2,y2,r2,x3,y3,r3)
        
    df2 = df2.append({'x1':x1, 'y1':y1, 'r1':r1, 
                      'x2':x2, 'y2':y2, 'r2':r2, 
                      'x3':x3, 'y3':y3, 'r3':r3, 
                      'seqnum1': option1, 'seqnum2': option2, 'seqnum3': option3,
                      'xTrue':xTrue, 'yTrue':yTrue, 
                      'xCalc':xCalc, 'yCalc':yCalc} , ignore_index=True)

df2.drop_duplicates(keep = 'first', inplace=True)

Calcul de la mse pour le calcul de la position

In [8]:
df2

,seqnum1,x1,y1,r1,seqnum2,x2,y2,r2,seqnum3,x3,y3,r3,xTrue,yTrue,xCalc,yCalc
0,6.0,-1.190,4.578,3.761,2.0,6.169,7.657,6.329,4.0,-3.962,7.931,7.962,2.120,2.793,2.120013,2.792880
6,7.0,2.120,2.793,0.000,11.0,-3.132,4.339,5.475,8.0,6.169,7.657,6.329,2.120,2.793,2.120072,2.792596
12,14.0,6.169,7.657,6.329,16.0,-3.962,7.931,7.962,13.0,2.120,2.793,0.000,2.120,2.793,2.120007,2.792650
18,20.0,6.169,7.657,6.329,24.0,-1.190,4.578,3.761,22.0,-3.962,7.931,7.962,2.120,2.793,2.120013,2.792880
24,30.0,-1.190,4.578,3.761,29.0,-3.132,4.339,5.475,25.0,2.120,2.793,0.000,2.120,2.793,2.119913,2.792055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918,3923.0,-3.132,4.339,4.639,3922.0,-3.962,7.931,2.978,3919.0,2.120,2.793,6.500,-1.034,8.476,-1.033939,8.476804
3924,3928.0,-3.962,7.931,2.978,3925.0,2.120,2.793,6.500,3930.0,-1.190,4.578,3.901,-1.034,8.476,-1.035722,8.474694
3930,3932.0,6.169,7.657,7.249,3934.0,-3.962,7.931,2.978,3935.0,-3.132,4.339,4.639,-1.034,8.476,-1.033768,8.476843
3936,3940.0,-3.962,7.931,2.978,3942.0,-1.190,4.578,3.901,3941.0,-3.132,4.339,4.639,-1.034,8.476,-1.032871,8.477051


In [ ]:
len(np.unique(df['seqnum']))

3946

In [ ]:
658*6

3948

Test sur les données avec changement d'orientation

In [ ]:
ecartX = (df2['xTrue']-df2['xCalc'])**2
ecartY = (df2['yTrue']-df2['yCalc'])**2

rmse = ((ecartX+ecartY)/4)**0.5
new_df2=df2.assign(RMSE=rmse)
new_df2

,seqnum1,x1,y1,r1,seqnum2,x2,y2,r2,seqnum3,x3,y3,r3,xTrue,yTrue,xCalc,yCalc,RMSE
0,1.0,2.0,2.0,0.0,2.0,6.0,7.0,6.0,5.0,-3.0,4.0,5.0,2.0,2.0,1.848485,2.621212,0.319711
6,8.0,6.0,7.0,6.0,12.0,-1.0,4.0,3.0,7.0,2.0,2.0,0.0,2.0,2.0,1.782609,2.673913,0.354054
12,13.0,2.0,2.0,0.0,17.0,-3.0,4.0,5.0,16.0,-3.0,7.0,7.0,2.0,2.0,1.400000,1.500000,0.390512
18,20.0,6.0,7.0,6.0,23.0,-3.0,4.0,5.0,22.0,-3.0,7.0,7.0,2.0,2.0,2.222222,1.500000,0.273579
24,28.0,-3.0,7.0,7.0,29.0,-3.0,4.0,5.0,30.0,-1.0,4.0,3.0,2.0,2.0,2.000000,1.500000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918,3922.0,-3.0,7.0,2.0,3919.0,2.0,2.0,6.0,3924.0,-1.0,4.0,3.0,-1.0,8.0,-1.600000,6.600000,0.761577
3924,3928.0,-3.0,7.0,2.0,3925.0,2.0,2.0,6.0,3930.0,-1.0,4.0,3.0,-1.0,8.0,-1.600000,6.600000,0.761577
3930,3931.0,2.0,2.0,6.0,3936.0,-1.0,4.0,3.0,3934.0,-3.0,7.0,2.0,-1.0,8.0,-1.600000,6.600000,0.761577
3936,3941.0,-3.0,4.0,4.0,3939.0,11.0,9.0,12.0,3938.0,6.0,7.0,7.0,-1.0,8.0,-2.000000,10.500000,1.346291
